# Linear Regression

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import feature_selection
import pandas.tseries
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

/Users/sijiaxiao/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
df_cap = pd.read_csv('T_UWWTPS.csv')
df_cap2 = df_cap.drop(df_cap.index[4332]) # remove abnormal point
df_temp = pd.read_csv('location_temperature.csv')
df_cap3 = df_cap2[['uwwLatitude', 'uwwLongitude']]
df_cleaned = pd.DataFrame(data = {'LoadEntering': df_cap2['uwwLoadEnteringUWWTP'],'Capacity': df_cap2['uwwCapacity'], 
                        'T': df_temp['temperature'],'NRemoval':df_cap2['uwwNRemoval'],'PRemoval':df_cap2['uwwPRemoval'],
                        'Longitude': df_cap3['uwwLongitude'], 'Latitude':df_cap3['uwwLatitude']})
df_no_missing = df_cleaned.dropna()
df_no_zeros = df_no_missing[df_no_missing.LoadEntering != 0]
df = df_no_zeros[df_no_zeros.Capacity != 0]

x = df[['LoadEntering', 'Longitude', 'Latitude']]
y = df['Capacity']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 1010)
df_train = pd.DataFrame(data = {'Capacity':y_train, 'LoadEntering':x_train['LoadEntering'], 
                        'Longitude':x_train['Longitude'], 'Latitude':x_train['Latitude']})

model = smf.ols("Capacity ~ LoadEntering", df_train)
result = model.fit()
y_predict = result.predict(x_test)
r_2 = r2_score(y_test, y_predict)

In [ ]:
# plot scatter points of training set and testing set
plt.subplots(figsize = (8, 5), dpi = 800)
plt.scatter(x_train.LoadEntering, y_train, c='tan', marker = '.', label = 'training data')
plt.scatter(x_test.LoadEntering, y_test, c='r', marker = '.', label = 'testing data')
plt.plot(x_test.LoadEntering, y_predict)
plt.legend()
plt.xlabel('Load Entering')
plt.ylabel('Capacity')
plt.title('Linear Regression')

In [ ]:
# studentized residual and leverage
influence = result.get_influence()
stu_residual = influence.resid_studentized_external
(cooks, p) = influence.cooks_distance
(dffits, p) = influence.dffits
leverage = influence.hat_matrix_diag
# high leverage points
fig, axes = plt.subplots(1, 2, figsize = (10, 4), dpi = 700)
axes[0].scatter(leverage, stu_residual, marker = '.')
axes[0].set_title('Studentized Residual vs. Leverage')
axes[0].set_xlabel('Leverage')
axes[0].set_ylabel('Studentized Residual')
axes[0].axhline(y = 0, ls = '--', linewidth = 0.7, c = 'black')
# outliers
axes[1].scatter(result.predict(), stu_residual, marker='.')
axes[1].set_title('Studentized Residual vs. Fitted Values')
axes[1].set_xlabel('Fitted Values')
axes[1].set_ylabel('Studentized Residual')
axes[1].axhline(y = 0, ls = '--', linewidth = 0.7, c = 'black')

In [ ]:
print(mean_squared_error(y_train, result.predict(x_train)))
print(mean_squared_error(y_test, result.predict(x_test)))

# Ridge Regression

In [ ]:
import xlrd
from sklearn.linear_model import Ridge

In [ ]:
train,test = train_test_split(df, test_size=0.1, random_state=1010)
#normalized data for Ridge / LASSO 
train_normalized=train/train.std()
test_normalized=test/test.std()

In [ ]:
heat_ridge=Ridge()
a=1e0
heat_ridge.set_params(alpha=a)
result2=heat_ridge.fit(train_normalized[['Latitude','LoadEntering','Longitude']],train_normalized.Capacity)

In [ ]:
plt.subplots(figsize = (10, 8), dpi =700)
plt.scatter(train_normalized.Capacity,heat_ridge.predict(train_normalized[['Latitude','LoadEntering','Longitude']]),
           label='training set')
plt.scatter(test_normalized.Capacity,heat_ridge.predict(test_normalized[['Latitude','LoadEntering','Longitude']])
            , c='r', label='testing set')
plt.plot([0,60], [0, 60], c='black',ls='--')
plt.xlabel('Normalized observed Capacity')
plt.ylabel('Normalized predicted capacity')
plt.title('Ridged Regression')
plt.xlim((0, 60))
plt.ylim((0, 60))
plt.legend()

In [ ]:
customer = pd.DataFrame(data = {'LoadEntering': [6200.0], 'Longitude': [17.0], 'Latitude': [47.0], 'NRemoval': [True], 'PRemoval':[True]})
heat_ridge.predict(customer[['Latitude','LoadEntering','Longitude']])

In [ ]:
r2_score(test_normalized.Capacity, heat_ridge.predict(test_normalized[['Latitude','LoadEntering','Longitude']]))

In [ ]:
# RR vs lambda (based on sklearn tutorial)
coefs = []
trainerror = []
testerror = []

# do you know what is happening here? 
lambdas = np.logspace(-6,6,200)
model=Ridge()

# loop over lambda values (strength of regularization)
for l in lambdas:
    model.set_params(alpha=l)
    model.fit(train_normalized[['Latitude','LoadEntering','Longitude']],train_normalized.Capacity)
    coefs.append(model.coef_)
    trainerror.append(mean_squared_error(train_normalized.Capacity,model.predict(
        train_normalized[['Latitude','LoadEntering','Longitude']])))
    testerror.append(mean_squared_error(test_normalized.Capacity,model.predict(
        test_normalized[['Latitude','LoadEntering','Longitude']])))

plt.figure(figsize=(10,3))
plt.subplot(121)
plt.plot(lambdas,coefs)
plt.xscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('coefs')
plt.title('RR coefs vs $\lambda$')
plt.subplot(122)
plt.plot(lambdas,trainerror,label='train error')
plt.plot(lambdas,testerror,label='test error')
plt.xscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('error')
plt.legend(loc=1)
plt.title('error vs $\lambda$')

* F_statistics is greater than 1 and p value is 0, which is a strong evidence to show that there is a relationship between LoadEntering and Capacity. $R^2$ is 0.867, which means 86.7% of training data can be interpreted in the regression line. 

In [ ]:
# studentized residual and leverage
influence = result.get_influence()
stu_residual = influence.resid_studentized_external
(cooks, p) = influence.cooks_distance
(dffits, p) = influence.dffits
leverage = influence.hat_matrix_diag
# high leverage points
fig, axes = plt.subplots(1, 2, figsize = (10, 4), dpi = 700)
axes[0].scatter(leverage, stu_residual, marker = '.')
axes[0].set_title('Studentized Residual vs. Leverage')
axes[0].set_xlabel('Leverage')
axes[0].set_ylabel('Studentized Residual')
axes[0].axhline(y = 0, ls = '--', linewidth = 0.7, c = 'black')
# outliers
axes[1].scatter(result.predict(), stu_residual, marker='.')
axes[1].set_title('Studentized Residual vs. Fitted Values')
axes[1].set_xlabel('Fitted Values')
axes[1].set_ylabel('Studentized Residual')
axes[1].axhline(y = 0, ls = '--', linewidth = 0.7, c = 'black')

In [ ]:
dfy = pd.DataFrame(data = {'A': [1, 2, 3, 4, 5, np.inf], 'B':[0, 5, 4, 3, 5, 1], 'C': [2, 4, 6, 8, 9, 4]})
dfy

In [ ]:
((dfy.B.all()) and (dfy.C.all())).all()

## Demo Section

In [ ]:
import linear_regression as lr

In [ ]:
df = lr.data_cleaning()

In [ ]:
r2_lr, filename_lr, r2_rr, mse_rr, filename_rr = lr.linear_regression_result()

In [ ]:
customer = pd.DataFrame(data = {'LoadEntering': [6200.0], 'Longitude': [47.0], 'Latitude': [47.0], 'NRemoval': [True], 'PRemoval':[True]})

In [ ]:
y_customer_lr, y_customer_rr = lr.customer_inter(customer, filename_lr, filename_rr)

In [ ]:
print('filename:      ', filename_lr, '\t',filename_rr)
print('prediction:    ',y_customer_lr.tolist()[0], '\t',y_customer_rr[0])
print('R^2:           ', r2_lr, '\t',r2_rr)
print('Normalized MSE:', mse_rr)

In [2]:
import nearest_n as nn

In [3]:
customer = pd.DataFrame(data = {'LoadEntering': [6200.0], 'Longitude': [47.0], 'Latitude': [47.0], 'NRemoval': [True], 'PRemoval':[True]})
df_NP = nn.NP_removal(customer)

In [12]:
df_NP.index = ['customer', 'NP-Removal', 'Non NP-Removal']
df_NP

,Latitude,LoadEntering,Longitude,NRemoval,PRemoval,Capacity
customer,47.00000,6200.0,47.0000,True,True,NaN
NP-Removal,55.83380,6200.0,24.9413,True,True,9400.0
NP-nonRemoval,44.98956,6206.0,26.2243,False,False,8292.0


In [18]:
import os.path
os.path.isfile("ridge_result.sav") 

True